In [1]:
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.compose import ColumnTransformer
import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

import os

import math


from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import datetime as dt

import warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # turn off deprecation warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


from xgboost import XGBRegressor
from sklearn.metrics import  mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import time

2023-05-21 17:37:11.767970: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 17:37:11.833727: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 17:37:11.835226: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 17:37:12.804295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv('../../data/latest_hour_final_btc.csv')

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)

scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)


# Start measuring the execution time
start_time = time.time()

# Create the 'future' column by shifting the 'closing_price' column 10 rows forward
df_normalized['future'] = df_normalized['close'].shift(-1)

df_normalized = df_normalized.dropna()


X = df_normalized.drop(columns='future', axis=1)  # Exclude the target column from features
y = df_normalized['future']  # Target column (the one you want to predict)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



my_pipeline = Pipeline([ ('xgbrg', XGBRegressor())   ])


param_grid = {
    "xgbrg__n_estimators": [10, 50, 100],
    
    "xgbrg__learning_rate": [0.01, 0.1, 0.5, 1],
} 


searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid)

searchCV.fit(X_train, y_train) 




# After fitting the GridSearchCV object
best_params = searchCV.best_params_

# Modify the best_params dictionary to match the correct parameter names
best_params = {
    'n_estimators': best_params['xgbrg__n_estimators'],
    'learning_rate': best_params['xgbrg__learning_rate']
}

# Create a new instance of the model with the best parameters
best_model = XGBRegressor(**best_params)

# Fit the best model on the entire training data
best_model.fit(X_train, y_train)
 
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print MSE and MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


Elapsed time: 80.88625979423523 seconds
Mean Squared Error (MSE): 0.00011649733790087549
Mean Absolute Error (MAE): 0.008316633596719516


In [3]:
df = pd.read_csv('../../data/latest_hour_final_btc.csv')

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)

scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)


# Start measuring the execution time
start_time = time.time()

# Create the 'future' column by shifting the 'closing_price' column 10 rows forward
df_normalized['future'] = df_normalized['close'].shift(-3)

df_normalized = df_normalized.dropna()


X = df_normalized.drop(columns='future', axis=1)  # Exclude the target column from features
y = df_normalized['future']  # Target column (the one you want to predict)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



my_pipeline = Pipeline([ ('xgbrg', XGBRegressor())   ])

param_grid = {
    "xgbrg__n_estimators": [10, 50, 100,  200],
    
    "xgbrg__learning_rate": [0.001,0.01, 0.1, 0.5, 1],
} 


searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid)

searchCV.fit(X_train, y_train)  

# After fitting the GridSearchCV object
best_params = searchCV.best_params_

# Modify the best_params dictionary to match the correct parameter names
best_params = {
    'n_estimators': best_params['xgbrg__n_estimators'],
    'learning_rate': best_params['xgbrg__learning_rate']
}

# Create a new instance of the model with the best parameters
best_model = XGBRegressor(**best_params)

# Fit the best model on the entire training data
best_model.fit(X_train, y_train)
 
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print MSE and MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


Elapsed time: 715.4679706096649 seconds
Mean Squared Error (MSE): 0.0007725368481799351
Mean Absolute Error (MAE): 0.019117578201101826


In [4]:
df = pd.read_csv('../../data/latest_hour_final_btc.csv')

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)

scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)


# Start measuring the execution time
start_time = time.time()

# Create the 'future' column by shifting the 'closing_price' column 10 rows forward
df_normalized['future'] = df_normalized['close'].shift(-6)

df_normalized = df_normalized.dropna()


X = df_normalized.drop(columns='future', axis=1)  # Exclude the target column from features
y = df_normalized['future']  # Target column (the one you want to predict)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



my_pipeline = Pipeline([ ('xgbrg', XGBRegressor())   ])

param_grid = {
    "xgbrg__n_estimators": [10, 50, 100,  200],
    
    "xgbrg__learning_rate": [0.001,0.01, 0.1, 0.5, 1],
} 


searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid)

searchCV.fit(X_train, y_train)  

# After fitting the GridSearchCV object
best_params = searchCV.best_params_

# Modify the best_params dictionary to match the correct parameter names
best_params = {
    'n_estimators': best_params['xgbrg__n_estimators'],
    'learning_rate': best_params['xgbrg__learning_rate']
}


# Create a new instance of the model with the best parameters
best_model = XGBRegressor(**best_params)

# Fit the best model on the entire training data
best_model.fit(X_train, y_train)
 
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print MSE and MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


Elapsed time: 210.12177991867065 seconds
Mean Squared Error (MSE): 0.0007879048958133687
Mean Absolute Error (MAE): 0.020796456555588802


In [5]:
df = pd.read_csv('../../data/latest_hour_final_btc.csv')

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)

scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)


# Start measuring the execution time
start_time = time.time()

# Create the 'future' column by shifting the 'closing_price' column 10 rows forward
df_normalized['future'] = df_normalized['close'].shift(-12)

df_normalized = df_normalized.dropna()


X = df_normalized.drop(columns='future', axis=1)  # Exclude the target column from features
y = df_normalized['future']  # Target column (the one you want to predict)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



my_pipeline = Pipeline([ ('xgbrg', XGBRegressor())   ])

param_grid = {
    "xgbrg__n_estimators": [10, 50, 100,  200],
    
    "xgbrg__learning_rate": [0.001,0.01, 0.1, 0.5, 1],
} 

searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid)

searchCV.fit(X_train, y_train)  

# After fitting the GridSearchCV object
best_params = searchCV.best_params_

# Modify the best_params dictionary to match the correct parameter names
best_params = {
    'n_estimators': best_params['xgbrg__n_estimators'],
    'learning_rate': best_params['xgbrg__learning_rate']
}

# Create a new instance of the model with the best parameters
best_model = XGBRegressor(**best_params)

# Fit the best model on the entire training data
best_model.fit(X_train, y_train)
 
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print MSE and MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


Elapsed time: 715.634092092514 seconds
Mean Squared Error (MSE): 0.0013122094029579695
Mean Absolute Error (MAE): 0.026207110374512586


In [5]:
df = pd.read_csv('../../data/latest_hour_final_btc.csv')

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)

scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)


# Start measuring the execution time
start_time = time.time()

# Create the 'future' column by shifting the 'closing_price' column 10 rows forward
df_normalized['future'] = df_normalized['close'].shift(-24)

df_normalized = df_normalized.dropna()


X = df_normalized.drop(columns='future', axis=1)  # Exclude the target column from features
y = df_normalized['future']  # Target column (the one you want to predict)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



my_pipeline = Pipeline([ ('xgbrg', XGBRegressor())   ])


param_grid = {
    "xgbrg__n_estimators": [10, 50, 100,  200],
    
    "xgbrg__learning_rate": [0.001,0.01, 0.1, 0.5, 1],
} 

print(1)
searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid)

searchCV.fit(X_train, y_train)  

# After fitting the GridSearchCV object
best_params = searchCV.best_params_

# Modify the best_params dictionary to match the correct parameter names
best_params = {
    'n_estimators': best_params['xgbrg__n_estimators'],
    'learning_rate': best_params['xgbrg__learning_rate']
}

# Create a new instance of the model with the best parameters
best_model = XGBRegressor(**best_params)

print(2)


# Fit the best model on the entire training data
best_model.fit(X_train, y_train)
 
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print MSE and MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


1
2
Elapsed time: 208.05149364471436 seconds
Mean Squared Error (MSE): 0.003528178606690366
Mean Absolute Error (MAE): 0.04326569163545041


In [6]:
df = pd.read_csv('../../data/latest_hour_final_btc.csv')

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)

scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)


# Start measuring the execution time
start_time = time.time()

# Create the 'future' column by shifting the 'closing_price' column 10 rows forward
df_normalized['future'] = df_normalized['close'].shift(-27)
df_normalized = df_normalized.dropna()


X = df_normalized.drop(columns='future', axis=1)  # Exclude the target column from features
y = df_normalized['future']  # Target column (the one you want to predict)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



my_pipeline = Pipeline([ ('xgbrg', XGBRegressor())   ])


param_grid = {
    "xgbrg__n_estimators": [10, 50, 100,  200],
    
    "xgbrg__learning_rate": [0.001,0.01, 0.1, 0.5, 1],
} 

print(1)
searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid)

searchCV.fit(X_train, y_train)  

# After fitting the GridSearchCV object
best_params = searchCV.best_params_

# Modify the best_params dictionary to match the correct parameter names
best_params = {
    'n_estimators': best_params['xgbrg__n_estimators'],
    'learning_rate': best_params['xgbrg__learning_rate']
}

# Create a new instance of the model with the best parameters
best_model = XGBRegressor(**best_params)

print(2)


# Fit the best model on the entire training data
best_model.fit(X_train, y_train)
 
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")

# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print MSE and MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


1
2
Elapsed time: 210.5539424419403 seconds
Mean Squared Error (MSE): 0.0025622061991426633
Mean Absolute Error (MAE): 0.03770257142652256
